# A4 : Logistic regression

---

## 1. $f(x_1, x_2) = w_1x_1 + w_2x_2 + w_0$

the cross entropy error is defined as:

$$
E(w_1, w_2, w_0) = -\sum_{x, y \in D} y\log(\sigma(f(x))) + (1 - y)\log(1 - \sigma(f(x)))

$$

where $\sigma(z) = \frac{1}{1 + e^{-z}}$


### 1.1 partial derivative of $E(w_1, w_2, w_0)$ with respect to $w_0, w_1, w_2$

- $\frac{\partial E}{\partial w_0} = -\sum_{x, y \in D} y(1 - \sigma(f(x))) - (1 - y)\sigma(f(x)) = -\sum_{x, y \in D} y - \sigma(f(x))$
- $\frac{\partial E}{\partial w_1} = -\sum_{x, y \in D} y(1 - \sigma(f(x)))x_1 - (1 - y)\sigma(f(x))x_1 = -\sum_{x, y \in D} (y - \sigma(f(x)))x_1$
- $\frac{\partial E}{\partial w_2} = -\sum_{x, y \in D} y(1 - \sigma(f(x)))x_2 - (1 - y)\sigma(f(x))x_2 = -\sum_{x, y \in D} (y - \sigma(f(x)))x_2$


### 1.2 code for logistic regression with f(x) = w1x1 + w2x2 + w0


In [3]:
import numpy as np
from scipy.special import expit as sigmoid

def calc_gradient_logistic(w, given_data):
    x = given_data[:, :-1]  # Extracting features (x1, x2)
    t = given_data[:, -1]   # Extracting target values
    z = np.dot(x, w) # w0 + w1 * x1 + w2 * x2
    y_pred = sigmoid(z) # 1 / (1 + np.exp(-z))
    error = y_pred - t # y_pred - t
    grad = np.dot(x.T, error) # x.T * error (dE/dw0 = x0 * error, dE/dw1 = x1 * error, dE/dw2 = x2 * error)
    return grad # [total_dE_dw0, total_dE_dw1, total_dE_dw2]

def gradient_descent_logistic(learning_rate, max_iter=1000):
    data = np.loadtxt("./data.txt", delimiter=",")
    x = np.c_[np.ones(data.shape[0]), data[:, :-1]]  # Adding bias term (x0 = 1)
    t = data[:, -1]                                   # Extracting target values
    w = np.random.rand(x.shape[1]) # Random initialization of w

    for _ in range(max_iter):
        grad = calc_gradient_logistic(w, np.c_[x, t])
        # stop when the gradient is small enough
        if np.linalg.norm(grad) < 1e-6:
            break
        w -= learning_rate * grad
    return w

learning_rate = 0.01
solution = gradient_descent_logistic(learning_rate)
print(f"Solution: w0 = {solution[0]}, w1 = {solution[1]}, w2 = {solution[2]}")
print(f"Best fit: f(x) = {solution[0]} + {solution[1]} * x1 + {solution[2]} * x2")

# Calculate the predicted probability
def predict_proba(x, w):
    z = np.dot(x, w)
    return sigmoid(z)

# Calculate cross entropy
def calculate_cross_entropy(y_pred, t):
    epsilon = 1e-200  # Small epsilon value to prevent taking logarithm of 0
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip values to avoid logarithm of 0 or 1
    return -t * np.log(y_pred) - (1 - t) * np.log(1 - y_pred)


# Classify new data points
def classify_new_data(x, w):
    y_pred = predict_proba(x, w)
    return 1 if y_pred >= 0.5 else 0

# Calculate cross entropy for (33, 81, 1) and (33, 81, 0)
x = np.array([1, 33, 81])
t = 1
y_pred = predict_proba(x, solution)
cross_entropy = calculate_cross_entropy(y_pred, t)
print(f"Cross entropy for (33, 81, 1): {cross_entropy}")

t = 0
cross_entropy = calculate_cross_entropy(y_pred, t)
print(f"Cross entropy for (33, 81, 0): {cross_entropy}")

# Classify (33, 81)
x = np.array([1, 33, 81])
classification = classify_new_data(x, solution)
print(f"Classification of (33, 81): {classification}")



Solution: w0 = 28.800713028060414, w1 = 157.85978452669949, w2 = -71.78437144329715
Best fit: f(x) = 28.800713028060414 + 157.85978452669949 * x1 + -71.78437144329715 * x2
Cross entropy for (33, 81, 1): 460.51701859880916
Cross entropy for (33, 81, 0): -0.0
Classification of (33, 81): 0


# 2. $f(x_1, x_2) = w_5x_{2}^{2} + w_4x_{1}^{2} + w_3x_2x_1 + w_2x_2 + w_1x_1 + w_0$

### 2.1 partial derivative of $E(w_0, w_1, w_2, w_3, w_4, w_5)$ with respect to $w_0, w_1, w_2, w_3, w_4, w_5$

- $\frac{\partial E}{\partial w_0} = -\sum_{x, y \in D} y(1 - \sigma(f(x))) - (1 - y)\sigma(f(x)) = -\sum_{x, y \in D} y - \sigma(f(x))$
- $\frac{\partial E}{\partial w_1} = -\sum_{x, y \in D} y(1 - \sigma(f(x)))x_1 - (1 - y)\sigma(f(x))x_1 = -\sum_{x, y \in D} (y - \sigma(f(x)))x_1$
- $\frac{\partial E}{\partial w_2} = -\sum_{x, y \in D} y(1 - \sigma(f(x)))x_2 - (1 - y)\sigma(f(x))x_2 = -\sum_{x, y \in D} (y - \sigma(f(x)))x_2$
- $\frac{\partial E}{\partial w_3} = -\sum_{x, y \in D} y(1 - \sigma(f(x)))x_2x_1 - (1 - y)\sigma(f(x))x_2x_1 = -\sum_{x, y \in D} (y - \sigma(f(x)))x_2x_1$
- $\frac{\partial E}{\partial w_4} = -\sum_{x, y \in D} y(1 - \sigma(f(x)))x_{1}^{2} - (1 - y)\sigma(f(x))x_{1}^{2} = -\sum_{x, y \in D} (y - \sigma(f(x)))x_{1}^{2}$
- $\frac{\partial E}{\partial w_5} = -\sum_{x, y \in D} y(1 - \sigma(f(x)))x_{2}^{2} - (1 - y)\sigma(f(x))x_{2}^{2} = -\sum_{x, y \in D} (y - \sigma(f(x)))x_{2}^{2}$

### 2.2 code for logistic regression with f(x) = $w_5x_{2}^{2} + w_4x_{1}^{2} + w_3x_2x_1 + w_2x_2 + w_1x_1 + w_0$


In [4]:
import numpy as np
from scipy.special import expit as sigmoid
np.seterr(divide = 'ignore') 

def calc_gradient_logistic(w, given_data):
    H = given_data[:, :-1]  # Extracting features (x1, x2, x1*x2, x1^2, x2^2)
    t = given_data[:, -1]   # Extracting target values
    z = np.dot(H, w) # Corrected variable name
    y_pred = sigmoid(z)
    error = y_pred - t
    grad = np.dot(H.T, error)
    return grad

def gradient_descent_logistic(learning_rate, max_iter=1000):
    data = np.loadtxt("./data.txt", delimiter=",")
    H = np.c_[
            np.ones(data.shape[0]), # Adding bias term (x0 = 1)
            data[:, 0], 
            data[:, 1], 
            data[:, 0] * data[:, 1], 
            data[:, 0] ** 2, 
            data[:, 1] ** 2
            ]
    t = data[:, -1]
    w = np.random.rand(H.shape[1])

    for _ in range(max_iter):
        grad = calc_gradient_logistic(w, np.c_[H, t])
        # stop when the gradient is small enough
        if np.linalg.norm(grad) < 1e-6:
            break
        w -= learning_rate * grad
    return w

learning_rate = 0.01
solution = gradient_descent_logistic(learning_rate)
print(f"""Solution: 
      w0 = {solution[0]}, w1 = {solution[1]}, w2 = {solution[2]},w3 = {solution[3]}, w4 = {solution[4]}, w5 = {solution[5]}""")
print(f"""Best fit: 
      f(x) = {solution[0]} + {solution[1]} * x1 + {solution[2]} * x2 +{solution[3]} * x1*x2 + {solution[4]} * x1^2 + {solution[5]} * x2^2""")


# Calculate cross entropy for (33, 81, 1) and (33, 81, 0)
x = np.array([1, 33, 81, 33 * 81, 33 ** 2, 81 ** 2])
t = 1
y_pred = predict_proba(x, solution)
cross_entropy = calculate_cross_entropy(y_pred, t)
print(f"Cross entropy for (33, 81, 1): {cross_entropy}")

t = 0
cross_entropy = calculate_cross_entropy(y_pred, t)
print(f"Cross entropy for (33, 81, 0): {cross_entropy}")

# Classify (33, 81)
x = np.array([1, 33, 81, 33 * 81, 33 ** 2, 81 ** 2])
classification = classify_new_data(x, solution)
print(f"Classification of (33, 81): {classification}")


Solution: 
      w0 = 5.606667383185794, w1 = -425.44074634160273, w2 = -993.8311589976877,w3 = 9985.91012429743, w4 = 10212.596406376351, w5 = -5805.620270135602
Best fit: 
      f(x) = 5.606667383185794 + -425.44074634160273 * x1 + -993.8311589976877 * x2 +9985.91012429743 * x1*x2 + 10212.596406376351 * x1^2 + -5805.620270135602 * x2^2
Cross entropy for (33, 81, 1): 460.51701859880916
Cross entropy for (33, 81, 0): -0.0
Classification of (33, 81): 0
